1. [前向分布算法](#前向分布算法)
2. [负梯度拟合](#负梯度拟合)
3. [损失函数](#损失函数)
4. [回归](#回归)
5. [分类](#分类)
6. [正则化](#正则化)
7. [优缺点](#优缺点)
8. [sklearn参数](#sklearn参数)
9. [应用场景](#应用场景)

# 前向分布算法
对加法模型$$f(x)=\sum_{m=1}^{M}\beta_mb(x;\gamma_m)$$ 在给定训练数据及损失函数L(y,f(x))的条件下, 模型的目标为经验风险最小化$$\mathop{\min}_{\beta_m,\gamma_m}\sum_{i=1}^{N}L\left(y_i,\sum_{m=1}^{M}\beta_mb(x_i;\gamma_m)\right)$$
前向分布算法的思路如加法模型一样, 从前往后,每一步只学习一个基函数及其系数, 逐步逼近优化目标函数

具体, 每一步只需要优化$$\mathop{\min}_{\beta_m,\gamma_m}\sum_{i=1}^{N}L(y_i,\beta b(x_i;\gamma))$$

>算法

输入: 训练数据集$T={(x_1,y_1),(x_2,y_2),...,(x_N,y_N)}, x_i\in \mathcal X \subseteq R^n, y\in \mathcal Y=\{-1,+1\}, 损失函数L(y,f(x)), 基函数集${b(x;$\gamma$)}

输出: 加法模型f(x)

- 1. 初始化$f_0(x)=0$
- 2. 对m = 1,2,...,M
    - a. 极小化损失函数$$(\beta_m, \gamma_m)=\mathop{\arg\min}_{\beta,\gamma}\sum_{i=1}^{N}L(y_i, f_{m-1}(x_i)+\beta b(x_i;\gamma))$$得到参数$\beta_m, \gamma_m$
    - b. 更新$$f_m(x) = f_{m-1}(x)+\beta_mb(x;\gamma_m)$$
- 3. 得到加法模型$$f(x) = f_M(x) = \sum_{m=1}^{M}\beta_mb(x;\gamma_m)$$

前向分布算法将同时求解m=1到M的所有参数$\beta_m,\gamma_m$的优化问题简化为逐次求解各个$\beta_m, \gamma_m的优化问题$

GBDT也是一种前向分布算法, 其基学习器为CART回归树.

# 负梯度拟合
对采用平方误差损失函数的提升树而言,损失$$L(y,f_{m-1}+T(x;\Theta_m))\\=[y-f_{m-1}-T(x;\Theta_m)]^2\\=[r-T(x;\Theta_m)]^2 \\ r = y-f_{m-1}(x)$$通过拟合数据的残差可以得到下一棵子树;

与提升树的残差拟合不同, GBDT是用负梯度来拟合每一轮损失的近似值, 来得到一棵CART回归树子树,

第m棵子树的第i个样本的损失函数的负梯度$$r_{m,i} = - \left[\frac{\partial L(y_i,f(x_i))}{\partial f(x_i)} \right]_{f(x)=f_{m-1}(x)}$$




# 损失函数
### 分类任务
>指数损失

$$L(y, f(x)) = exp(-yf(x))$$

>对数损失

$$L(y, f(x)) = -\sum_{k=1}^{K}y_klogp_k(x)$$$$


### 回归任务
> MSE

$$L(y,f(x)) = (y-f(x))^2$$

> MAE

$$L(y, f(x)) = |y-f(x)|$$

> Huber损失

$$L(y,f(x)) = \left\{\begin{aligned} \frac{1}{2}(y-f(x))^2 && |y-f(x)|\leq \delta \\ \delta(|y-f(x)|-\frac{\delta}{2}) && |y-f(x)|>\delta \end{aligned}\right.$$

Hub损失是MAE与MSE的折中, 对远离中心的异常点采用MAE, 中心点采用MSE, 一般用分位数点来度量中心的界限

>分位数损失

$$L(y, f(x)) = \sum_{y\geq f(x)}\theta |y-f(ax)| + \sum_{y<f(x)}(1-\theta) |y-f(x)|$$

其中$\theta$为分位数, 需提前指定

Huber损失与分位数损失主要用来减少异常点对损失函数的影响



# [回归](https://www.cnblogs.com/zengzhen94/p/8744970.html)

输入: 训练数据集$T={(x_1,y_1),(x_2,y_2),...,(x_N,y_N)},  损失函数L(y,f(x))$

输出: 加法模型f(x)

- 1. 初始化$$f_0(x) = \arg\min_c\sum_{i=1}^{N}L(y_i, c)$$
- 2. 对m = 1,2,...,M
    - a. 计算每一个样本的负梯度$$r_{m,i} = - \left[\frac{\partial L(y_i,f(x_i))}{\partial f(x_i)} \right]_{f(x)=f_{m-1}(x)}$$
    - b. 用$\{(x_i, r_{m,i})\}_{i=1,2,...N}Pair训练出第m棵子树T_m, 对应的叶结点划分区域为R_{m,j},j=1,2,..,J$
    - c. 计算子树每个结点的值$$c_{m,j}=\arg\min_c\sum_{x_i\in R_{m,j}} L(y_i, f_{m-1}(x_i)+c)$$
    - d. 生成新的子树$$f_m(x)=f_{m-1}(x) + \sum_{j=1}^{J}c_{m,j}I(x\in R_{m,j})$$

GBDT将回归树和提升树的算法结合, 在计算结点输出时, 如果为平方损失函数，则与回归树一致，取其均值;如果是其他损失函数，则需要具体进行求解。具体而言，就是取导数为零来解等式。

# 分类
对于分类情况，由于样本输出不是连续的值，而是离散的类别，导致我们无法直接从输出类别去拟合类别输出的误差。

主要有两个方法
- 1. 用指数损失，此时GBDT退化为Adaboost算法。

- 2. 用对数似然损失,用类别的预测概率值和真实概率值的差来拟合损失。

### 二分类
使用指数损失$$L(y,f(x)) = log(1+ exp(-yf(x))$$

此时负梯度$$r_{m,i} = - \left[\frac{\partial L(y_i,f(x_i))}{\partial f(x_i)} \right]_{f(x)=f_{m-1}(x)} \\= \frac{y_i}{1+exp(y_if_{m-1}(x_i))$$

结点输出值应为$$c_{m,j}=\arg\min_c\sum_{x_i\in R_{m,j}}log(1+exp(-y_i(f_{m-1}(x_i) + c)))$$由于次式在实际运算中难以优化, 通常采用近似解$$c_{m,j} = \frac{\sum_{x_i\in R_{m,j}}r_{m,i}}{\sum_{x_i\in R_{m,j}}|r_{m,i}|(1-|r_{m,i})}$$

### 多分类
使用对数损失$$L(y, f(x)) = -\sum_{k=1}^{K}y_klogp_k(x)$$

此时针对每个类别计算其负梯度$$r_{m,i,l} = - \left[\frac{\partial L(y_i,f(x_i))}{\partial f(x_i)} \right]_{f(x)=f_{m-1,l}(x)} \\= y_{i,l}-p_{m,l}(x_i)$$

结点输出值近似为$$c_{m,l,j}= \frac{K-1}{K}\frac{\sum_{x_i\in R_{m,l,j}}r_{m,i,l}}{\sum_{x_i\in R_{m,l,j}}|r_{m,l,i}|(1-|r_{m,l,i})}$$

# 正则化
>subsample

按预定的比例, 无放回抽样, 

> 子树加权

对每个子树给予一个0到1之间权值, 所有权值的和为1; 每一个预测结果相当于是对所有子树预测结果进行加权平均

> CART剪枝

对每一个子树进行树方法的正则

>Regularized Learning Objective

将树模型的复杂度作为正则项显式地加进优化目标里，这也是XGBoost实现的独到之处$$L_r(y,f(x))=L(y,f(x))+\gamma T_{leaf} + \lambda \|w|\|^2$$

>Dropout

类似DeepLearning 的 Dropout ，其具体可以参考文献[8]。通俗地讲，每次新加一棵树，这棵树要拟合的并不是之前全部树ensemble后的残差，而是随机抽取的一些树ensemble。

# 优缺点
>优点

- 1. 可处理多种类型的数据
- 2. 可采用Huber损失和分位数损失,对异常值较多的数据效果非常好
- 3. 在相对较少的调参时间下，预测的准确度较高。 

>缺点

- 1. 由于弱学习器之间存在依赖关系，难以并行训练数据, 通过自采样的SGBT来达到部分并行.


# 应用场景
搜索引擎排序应用 RankNet



# sklearn参数
|参数|用途
|:-:|:-:
|loss|损失函数度量，有对数似然损失deviance和指数损失函数exponential两种，默认是deviance，即对数似然损失，如果使用指数损失函数，则相当于Adaboost模型。
|criterion| 样本集的切分策略，决策树中也有这个参数，但是两个参数值不一样，这里的参数值主要有friedman_mse、mse和mae3个，分别对应friedman最小平方误差、最小平方误差和平均绝对值误差，friedman最小平方误差是最小平方误差的近似。
|subsample|采样比例，这里的采样和bagging的采样不是一个概念，这里的采样是指选取多少比例的数据集利用决策树基模型去boosting，默认是1.0，即在全量数据集上利用决策树去boosting。
|warm_start|“暖启动”，默认值是False，即关闭状态，如果打开则表示，使用先前调试好的模型，在该模型的基础上继续boosting，如果关闭，则表示在样本集上从新训练一个新的基模型，且在该模型的基础上进行boosting。
|max_features=None|在寻找最佳分割点要考虑的特征数量auto全选/sqrt开方/log2对数/None全选/int自定义几个/float百分比
|min_impurity_split=1e-7|停止分裂叶子节点的阈值
